In [6]:
'''
Handy Ni
Parking Mapping and Data handling file for the time being
'''

from branca import colormap
from folium.features import GeoJsonTooltip
import numpy as np
import pandas as pd
from pandas.core.dtypes.missing import na_value_for_dtype
import pandasql as psql
import matplotlib.pyplot as plt
import geojson as gs
import geopandas as gpd
import datetime as dt
import folium
import json 
import re
from geopy.geocoders import Nominatim


def roundTime(df, timeCol): #time rounding function
    tmp = ''
    for i in range(len(df)):
        tmp = df.loc[i, timeCol]
        tmp = tmp[-1:]
        x = round(int(df.loc[i, timeCol][:-1]), -2)
        df.loc[i, timeCol] = str(x) + tmp
    return df

def streetNameDf(df): #returns a df containing occurences of parking violations per street 
    query = 'SELECT "Street Name", COUNT("Street Name") as Count FROM df GROUP BY "Street Name"'
    outDf = psql.sqldf(query)
    return outDf

def countOfUniqueIds(df):
    query = 'SELECT COUNT(DISTINCT("Plate ID")) FROM df'
    outDf = psql.sqldf(query)
    return outDf

def cleanStreet(str):
    str = str.upper()
    if('AVENUE' in str):
        str = str.replace('AVENUE', 'AVE')
    if('STREET' in str):
        str = str.replace('STREET', 'ST')
    str = (re.compile(r"(?<=\d)(ND|RD|TH)").sub("", str))

    return str

def cleanStreetHelper(df, str): #name cleaning for usage for comparisons
    df[str] = df[str].str.upper()
    if('AVE' in df.loc[df[str]]):
        print('big')
        df[str] = df[str].replace('AVENUE', 'AVE')
    if('STREET' in df.loc[df[str]]):
        df[str] = df[str].replace('STREET', 'ST')
    for i in range(len(df[str])):
        if(df[str][i].isdigit()):
            if(df[str][i+1].isalpha()):
                new = df[str][i+1] + df[str][i+2]
                df[str].replace(new, '')
    return df

def pVfromStreet(df, streetname): #Returns all the rows where StreetName = streetname as a dataframe used for counting Parking Violations upon a street input
    streetname = streetname.upper()
    out = df[df['Street Name'] == streetname]
    return out

def returnZipCode(streetname): #Returns a zipcode when given an address
    geolocator = Nominatim(user_agent="geoapiExercises")
    place = streetname
    place = place + ' NYC'
    location = geolocator.geocode(place)
    if(location is None):
        return
    data = location.raw
    out_data = data['display_name'].split()
    if((out_data[-3])[:5].isdecimal()):
        return (out_data[-3])[:5]
    else: 
        return

def zipSeries(df): ##returns all zips in a series/ Cleaned
    cp = pd.Series(dtype=object)
    cp = df['Street Name']
    cp = cp.apply(returnZipCode)
    cp = cp.dropna()
    return cp

def returnPercentDf(zip):#returns a zip code df with counts for parsing and mapping 
    df = pd.DataFrame()
    df['Zip Code'] = zip
    query = 'SELECT "Zip Code", COUNT("Zip Code") as "Parking Violations Count" FROM df GROUP BY "Zip Code"'
    outDf = psql.sqldf(query)
    return outDf

def combineDups(df):
    df = df.groupby(['Roadway Name']).sum()
    return df

def makeTrafficSumDf(df):
    df1 = df.sum(axis=1, skipna=True)
    return df1

def getCoords(geojson, zipcode):
    print(zipcode)
    singular = geojson['postalcode']
    lat = []
    long = []
    for i in range(len(singular)):
        if(str(singular[i]) == str(zipcode)):
            new = str(geojson['geometry'][i])
            new = new.replace('MULTIPOLYGON ', '')
            new = new.replace('(', '')
            new = new.replace(')', '')
            new = new.replace(',', '')
            arr = new.split(' ')
            break
    for i in range(len(arr)):
        if(i % 2 != 0):
            lat.append(float(arr[i]))
        else:
            long.append(float(arr[i]))
    lat = sum(lat)/len(lat)
    long = sum(long)/len(long) * -1

    return lat, long 

def applyToolTips(df, geojson):
    tooltip = []
    singular = geojson['postalcode']
    zip = df['Zip Code'].apply(str)
    for i in singular:
        bool1 = zip.str.contains(i)
        if(bool1.sum() > 0):
            #print(df.loc[df['Zip Code'] == i])
            tooltip.append(df.loc[df['Zip Code'] == i, 'Parking Violations Count'].iloc[0])
        else:
            tooltip.append(np.nan)
    
    geojson['tooltip'] = tooltip

    return geojson 

def makeZipcodeMap(data):
    zipCodes = zipSeries(parkingDf)
    makeZipDf = returnPercentDf(zipCodes)
    data = applyToolTips(makeZipDf, data)
    map = folium.Map(location=[40.693943, -73.985880], default_zoom_start=15)
    choropleth = folium.Choropleth(
        geo_data=data,
        name="cholorpleth",
        fill_opacity=0.7,
        line_opacity=0.4,
        highlight=True,
        fill_color='#7851A9',
        key_on="features.properties.postalcode",
    ).add_to(map)
    #for i in range(len(df)):=
    choropleth.geojson.add_child(GeoJsonTooltip(['po_name', 'postalcode', 'tooltip'], aliases=['Name (If available): ', 'Zip Code: ', 'Parking Violation Count: '], labels=True))
    return map



#folium.Marker(location=[40.69153, -73.95605], popup="pp").add_to(map)

# Handling Inputs / Testing Inputs
#TVC = pd.read_csv('Traffic_Volume_Counts__2014-2019_.csv')

# <<< INITIALIZING DATA >>>
x = input('\033[1;32;40mWould you like to run default (0) or single out a street (1)?\n\033[0;37;40m')
if(x == '0'):
    PViol = pd.read_csv('Pviol.csv')
    query = 'SELECT "Street Name", "Issue Date", "Plate Type" FROM PViol'
    parkingDf = psql.sqldf(query)  
    data = gpd.read_file('Zip_code.geojson')
    parkingDf = cleanStreetHelper(parkingDf, 'Street Name')
    makeZipcodeMap(data).save('map.html')
elif(x == '1'):
    y = input('\033[1;32;40mPlease enter the street you are interested in\n\033[0;37;40m')
    y = cleanStreet(y)
    PViol = pd.read_csv('Pviol.csv')
    query = 'SELECT "Street Name", "Issue Date", "Plate Type" FROM PViol'
    parkingDf = psql.sqldf(query)  
    data = gpd.read_file('Zip_code.geojson')
    parkingDf['Street Name'] = parkingDf['Street Name'].apply(cleanStreet)
    traffic = pd.read_csv('TVC.csv')
    traffic = traffic.drop(columns=['ID', 'Segment ID', 'Direction', 'Date'])
    traffic['Roadway Name'] = traffic['Roadway Name'].apply(cleanStreet)
    traffic = combineDups(traffic)
    sumOfTraffic = makeTrafficSumDf(traffic)
    if(y in sumOfTraffic.index):
        #print(sumOfTraffic.loc[y]) #use for coordinates on marker
        parkingDf = streetNameDf(parkingDf)
        r1 = 0
        if(parkingDf['Street Name'].str.contains(y).any()):
            r1 = parkingDf.loc[parkingDf['Street Name'] == y, 'Count'].iloc[0] 
        ratio = str(r1) + "Parking Violation : " + str(sumOfTraffic.loc[y]) + " Daily Average Traffic" #gets our ratio and returns it as a string for using it in our tooltip 
        tmp = y
        postal = returnZipCode(tmp) #gets our zipcode 
        if(returnZipCode(tmp) == None): #gets us the marker coords using zipcode
            print('Could not fetch Zip Code, Defaulting to NYC coords')
            lat, long = 40.7128, -74.0060
        else:   
            lat, long = getCoords(data, 11225)
        map = makeZipcodeMap(data)
        folium.Marker(location=[lat, long], popup=ratio).add_to(map)
        makeZipcodeMap(data).save('map.html')

    else: 
        print('\033[1;33;40mStreet not found in dataset, defaulting to default map print\n\033[0;37;40m')

    #zipCodes = zipSeries(parkingDf)
    #makeZipDf = returnPercentDf(zipCodes)
    #-----------------------------------------------------
    #makeZipDf = pd.read_csv('zipCodes.csv')
    #makeZipDf['Zip Code'] = makeZipDf['Zip Code'].apply(str)
    #data = applyToolTips(makeZipDf, data)
    #map = makeZipcodeMap(data)
    #map.save('map.html')

    




#parkingDf = roundTime(parkingDf, 'Violation Time')
#countOfStreets = streetNameDf(parkingDf

#<<< FUNCTION CALLS >>>
#parkingDf = cleanStreetHelper(parkingDf)
#zipCodes = zipSeries(parkingDf)


#makeZipDf = returnPercentDf(zipCodes)
#makeZipDf = pd.read_csv('zipCodes.csv')
#makeZipDf['Zip Code'] = makeZipDf['Zip Code'].apply(str)
#data = applyToolTips(makeZipDf, data)

#makeZipcodeMap(data).save('map.html')









11225


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=E+34+ST+NYC&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))